In [ ]:
import lithops
import boto3
from insult_filter_batch import insult_filter_batch


def dynamic_scaler():
    λ = 5
    T = 0.5
    Tr = 2
    C = 10

    sqs = boto3.client('sqs', region_name='us-east-1')
    queue_url = 'https://sqs.us-east-1.amazonaws.com/164573654608/InsultTextQueue'

    resp = sqs.get_queue_attributes(
        QueueUrl=queue_url,
        AttributeNames=['ApproximateNumberOfMessages']
    )

    backlog = int(resp['Attributes']['ApproximateNumberOfMessages'])
    print(f"📦 Mensajes en cola: {backlog}")

    if backlog == 0:
        print("✔️ Nada que procesar")
        return

    N = int((λ * T + backlog / Tr) / C)
    N = max(1, N)

    print(f"🚀 Lanzando {N} funciones con Lithops...")

    fexec = lithops.FunctionExecutor()
    fexec.map(insult_filter_batch, [None] * N)
    results = fexec.get_result()

    for i, r in enumerate(results):
        print(f"\n✅ Worker {i+1}:")
        if isinstance(r, list):
            for line in r:
                print("→", line)
        else:
            print(r)


In [ ]:
dynamic_scaler()
